<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/Precipitation-Trend-Monitoring/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xee -q
!pip install pymannkendall -q

In [ ]:
# Core libraries
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Geospatial libraries
import ee
import geemap
import xee
import xarray as xr
import geopandas as gpd
from shapely.geometry import MultiPolygon

# Statistical tools
import pymannkendall as mk

# Utilities
import requests
from io import BytesIO
import matplotlib.image as mpimg
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
ee.Authenticate()
ee.Initialize(project="Your google earth engine project name", opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Parameters
start_time = '1980-01'
end_time = '2019-12-31'
country_name = 'Italy'
Dataset_name="ECMWF/ERA5/MONTHLY"
Target_band='total_precipitation'
Variable_name = 'Precipitation'
Data_resolution = 27830
Output_unit ='mm'
scale_factor= 1
offset_factor = 0
unit_conversion_factor = 1000
correction_factor =1
operation = 'Average'

country = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017") \
    .filter(ee.Filter.eq('country_na', country_name))
roi = country.geometry()

collection = ee.ImageCollection(Dataset_name) \
    .select(Target_band) \
    .filterDate(start_time, end_time) \
    .filterBounds(roi)


def data_conversion(image):
    celsius = (image.multiply(unit_conversion_factor).multiply(scale_factor).add(offset_factor)).multiply(correction_factor)
    return celsius.copyProperties(image, image.propertyNames())

collection_converted = collection.map(data_conversion)

def add_year(image):
    year = ee.Date(image.get('system:time_start')).get('year')
    return image.set('year', year)

collection_with_year = collection_converted.map(add_year)

years = collection_with_year.aggregate_array('year').distinct().sort().getInfo()

yearly_images = []

for year in years:
    year_num = ee.Number(year)
    yearly_image = collection_with_year.filter(ee.Filter.eq('year', year_num)).sum()
    yearly_image = yearly_image.clip(roi).set({'year': year_num})
    yearly_images.append(yearly_image)

yearly_collection = ee.ImageCollection(yearly_images)

min_image = yearly_collection.reduce(ee.Reducer.min())
max_image = yearly_collection.reduce(ee.Reducer.max())

min_band = min_image.bandNames().get(0)
max_band = max_image.bandNames().get(0)

min_val = min_image.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=roi,
    scale=Data_resolution,
    maxPixels=1e13
).get(min_band)

max_val = max_image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=roi,
    scale=Data_resolution,
    maxPixels=1e13
).get(max_band)

min_val = min_val.getInfo()
max_val = max_val.getInfo()

palette = ['#f7fbff', '#c6dbef', '#6baed6', '#2171b5', '#08306b']
vis_params = {
    'min': min_val,
    'max': max_val/2,
    'palette': palette
}

Map = geemap.Map()

Map.addLayer(roi, {'color': 'black', 'width': 2}, f'{country_name} Boundary')

for i, year in enumerate(years):
    image = yearly_images[i]
    layer_name = f"{Variable_name} {int(year)}"
    Map.addLayer(image, vis_params, layer_name)

num_groups = 5

step_size = (max_val - min_val) / (num_groups - 1)

legend_labels = [f"{round(min_val + step_size * i)} - {round(min_val + step_size * (i + 1))}" for i in range(num_groups)]

legend_colors = [palette[i] for i in range(num_groups)]

legend_dict = dict(zip(legend_labels, legend_colors))

Map.add_legend(
    legend_title="",
    legend_dict=legend_dict,
    position="bottomleft",
    legend_font_size=32,
    legend_key_width=40,
    legend_key_height=40
)

Map.centerObject(roi,zoom=6)
Map


In [ ]:
n_years = len(years)
cols = 4
rows = math.ceil(n_years / cols)

fig, axs = plt.subplots(rows, cols, figsize=(20, rows * 4))
axs = axs.flatten()

for idx, year in enumerate(years):
    try:
        image_for_year = yearly_images[idx]

        url = image_for_year.getThumbURL({
            'min': min_val,
            'max': max_val/2,
            'dimensions': 512,
            'region': roi,
            'palette': palette,
            'format': 'png'
        })

        response = requests.get(url)
        img_data = mpimg.imread(BytesIO(response.content), format='PNG')

        axs[idx].imshow(img_data)

        axs[idx].set_title(f'Year {year}', fontsize=14)

        axs[idx].tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)

        for spine in axs[idx].spines.values():
            spine.set_visible(False)

    except Exception as e:
        print(f"Error loading image for year {year}: {e}")
        axs[idx].axis('off')

for i in range(len(years), len(axs)):
    fig.delaxes(axs[i])

plt.tight_layout()


plt.savefig('Plots.png', dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
average_data_per_year = []
model_data = {}

for year in years:
    year_num = ee.Number(year)
    yearly_image = collection_with_year.filter(ee.Filter.eq('year', year_num)).sum()
    yearly_image = yearly_image.clip(roi).set({'year': year_num})


    average_data = yearly_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=Data_resolution,
        maxPixels=1e13
    ).get(Target_band)

    average_data = average_data.getInfo()
    average_data_per_year.append(average_data)

    model_data[year] = average_data


years_np = np.array(years)
data_np = np.array(average_data_per_year)


z = np.polyfit(years_np, data_np, 1)
p = np.poly1d(z)
trend_line = p(years_np)


max_val = data_np.max()
min_val = data_np.min()
max_year = years_np[data_np.argmax()]
min_year = years_np[data_np.argmin()]

years_np = np.array(years)
temps_np = np.array(average_data_per_year)

z = np.polyfit(years_np, temps_np, 1)
p = np.poly1d(z)
trend_line = p(years_np)

plt.figure(figsize=(14, 4))

plt.plot(
    years_np,
    temps_np,
    marker='o',
    linestyle='-',
    color='#1f77b4',
    linewidth=2,
    markersize=6,
    label=f'{operation} {Variable_name} ({Output_unit})'
)

plt.plot(
    years_np,
    trend_line,
    linestyle='--',
    color='orange',
    linewidth=2,
    label=f'Trend Line (slope = {z[0]:.2f} ({Output_unit}/year))'
)

plt.title(f'{operation} {Variable_name} ({Output_unit}) Over the Years', fontsize=16)
plt.xlabel('Year', fontsize=13)
plt.ylabel(f'{Variable_name} {Output_unit}', fontsize=13)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(years_np, rotation=45)
plt.legend()
plt.tight_layout()

plt.savefig('Trend_Chart.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
ds = xr.open_dataset(collection, engine = 'ee', crs = 'EPSG:4326', scale = 0.27, geometry = roi)
def mk_test(img):
  test = mk.original_test(img)
  trend = test.trend
  trend_class = {'increasing':1, 'decreasing': -1, 'no_trend':0}
  trend_reclass = trend_class.get(trend, 0)
  score = test.s
  return trend_reclass, score


change_trend, change_score = xr.apply_ufunc(
    mk_test,
    ds[Target_band],
    input_core_dims = [['time']],
    output_core_dims = [[],[]],
    dask = 'allowed',
    vectorize = True,
    output_dtypes = ['int','float64']
)

change_trend = change_trend.rename('trend')
change_score = change_score.rename('score') * 0.0001   # just to normalize the scallbar
iran_geojson = country.geometry().getInfo()
iran_geojson_dict = json.dumps(iran_geojson)
iran_gdf = gpd.read_file(iran_geojson_dict)


min_lon, min_lat, max_lon, max_lat = iran_gdf.total_bounds

fig, axes = plt.subplots(2, 1, figsize=(16, 10), constrained_layout=False)
titles = ['Trend', 'Score',]
datasets = [change_trend, change_score]
cmaps = ['coolwarm', 'viridis']
vmins = [-1, None, None]
vmaxs = [1, None, 0.05]


cmap_score = mcolors.LinearSegmentedColormap.from_list('green_orange_red', ['green', 'orange', 'red'])


cmap_pvalue = mcolors.LinearSegmentedColormap.from_list('reverse_pvalue', ['pink', 'white', 'grey'])

cmap_trend = mcolors.ListedColormap(['blue', 'gray', 'red'])


norm_trend = mcolors.BoundaryNorm(boundaries=[-1,-0.5,0.5, 1], ncolors=3)

for idx, (ax, data, title, cmap, vmin, vmax) in enumerate(zip(axes, datasets, titles, cmaps, vmins, vmaxs)):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    if title == 'Score':
        im = data.plot(
            x='lon', y='lat', ax=ax, add_colorbar=False,
            cmap=cmap_score, vmin=0, vmax=1, robust=True
        )
        ax.set_title('Intensity')
    elif title == 'P-value':
        im = data.plot(
            x='lon', y='lat', ax=ax, add_colorbar=False,
            cmap=cmap_pvalue, vmin=0, vmax=0.05, robust=True
        )
        ax.set_title('Accuracy')
    else:
        im = data.plot(
            x='lon', y='lat', ax=ax, add_colorbar=False,
            cmap=cmap_trend, norm=norm_trend, robust=True
        )
        ax.set_title('Trend')

    iran_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=0.8)


    cbar = plt.colorbar(im, cax=cax, cmap=cmap)

    if title == 'Trend':
        cbar.set_ticks([-1, 0, 1])
        cbar.set_ticklabels(['Decreasing', 'No trend', 'Increasing'])
    elif title == 'Score':
        cbar.set_ticks([0, 0.5, 1])
        cbar.set_ticklabels(['Low', 'Moderate', 'High'])

    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)


plt.subplots_adjust(hspace=0.3)


plt.savefig('Trend_Plot.png', dpi=600, bbox_inches='tight')


plt.show()
